# Genetic Algorithm Iteration 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
def initial_population(bounds,step):
    """
    Generate randomized input data
    
    Parameters:
        bounds (list): Bounds for the input data, where each element is a tuple (min, max) for a dimension.
        step (float): The number of values for each parameters

    Returns:
        input (numpy.ndarray): Input data to be analyzed, where each row is a [mass, CG] value.
    """
        
    # Consider the bounds on mass and CG
    mass_min,mass_max,cg_min,cg_max = bounds[0][0],bounds[0][1],bounds[1][0],bounds[1][1]
    
    # Generate random input data
    mass = [random.uniform(mass_min, mass_max) for _ in range(step)]
    CG = [random.uniform(cg_min, cg_max) for _ in range(step)]
    input = np.array([(m, c) for m in mass for c in CG]).reshape((step*step, 2))
    #print("grid(mass, CG): ", input)
    return input

In [3]:
def simulation_data(input,step):
    """
    Simulate input to get output data.

    Parameters:
        input (numpy.ndarray): Input data to be analyzed, where each row is a [mass, CG] value.

    Returns:
        output (numpy.ndarray): Simulated output data, where each row is a [load_factor, velocity] value.
    """
    # Simulate output data using automation code
    
    # Generate simulated output data
    load_factor = np.random.uniform(-0.5,1.5,size=step*step)
    velocity = np.random.uniform(250,450,size=step*step)
    output = np.concatenate((load_factor.reshape(-1, 1), velocity.reshape(-1, 1)), axis=1)
    #print("simulated output(Load_factor,velocity): ", output)
    
    return output

In [4]:
def overall_data(input, output):
    """
    Create a dictionary of simulated data from input and output arrays.

    Parameters:
        input (numpy.ndarray): Input data used to simulate output data.
        output (numpy.ndarray): Simulated output data corresponding to the input data.

    Returns:
        sim_data (dict): Dictionary of simulated data, where keys are (mass, CG) tuples and values are (load_factor, velocity) tuples.
    """
    sim_data = {}
    
    # Iterate through each input/output pair and add it to the dictionary
    for i in range(len(input)):
        input_key = (input[i][0], input[i][1])
        output_value = (output[i][0], output[i][1])
        sim_data[str(input_key)] = str(output_value)
    
    #print("Simulated data(mass,CG = load_factor,velocity):",sim_data)
    
    # Return the dictionary of simulated data
    return sim_data

In [5]:
def separate_input_data(sim_data):
    """
    Separate input data from a dictionary of simulated data.

    Parameters:
        sim_data (dict): Dictionary of simulated data, where keys are (mass, CG) tuples and values are (load_factor, velocity) tuples.

    Returns:
        mass (list): List of masses from the input data.
        CG (list): List of center of gravities (CG) from the input data.
        load_factor (list): List of load factors from the simulated output data.
        velocity (list): List of velocities from the simulated output data.
    """
    # Extract the mass and CG values from the keys of the dictionary
    mass = [eval(k)[0] for k in sim_data.keys()]
    CG = [eval(k)[1] for k in sim_data.keys()]

    # Extract the load factor and velocity values from the values of the dictionary
    load_factor = [eval(i)[0] for i in sim_data.values()]
    velocity = [eval(i)[1] for i in sim_data.values()]

    # Return the separated input data as separate lists
    return mass, CG, load_factor, velocity


In [6]:
def objective_function(mass, CG, load_factor, velocity):
    """
    Calculate the cost of a design based on load factor and velocity.

    Parameters:
        mass (float): The mass of the design.
        CG (float): The center of gravity (CG) of the design.
        load_factor (float): The load factor of the design.
        velocity (float): The velocity of the design.

    Returns:
        cost (float): The cost of the design based on load factor and velocity.
        fitness_scores (float): The fitness values is the inverse of cost in order to maximize the cost
    """
    # Set the limit values for load factor and velocity
    load_factor_limit = 5
    velocity_limit = 700

    # Calculate the cost of the design based on load factor and velocity
    # Cost is calculated as the sum of the differences between the actual and limit values for load factor and velocity
    cost = (load_factor_limit - load_factor) + (velocity - velocity_limit)
    fitness_scores = 1/(cost + 1e-9)
    
    # Return the fitness of the design
    return cost,fitness_scores

In [7]:
def tournament_selection(population, fitness_scores, n_sel):
    """
    Perform tournament selection on a population of individuals.

    Args:
        population (list): A list of individuals to select from.
        fitness_scores (list): A list of fitness scores for each individual in the population.
        n_sel (float): The number of individuals to select.

    Returns:
        The selected individual with the highest fitness score.
    """

    # Select an initial individual at random
    selected_index = np.random.randint(0, len(population))

    # Perform tournament selection to choose the best individual
    for _ in range(n_sel - 1):
        # Randomly select another individual from the population
        index = np.random.randint(0, len(population))

        # Compare the fitness scores of the two individuals
        if fitness_scores[index] < fitness_scores[selected_index]:
            # If the new individual has a better fitness score, select it
            selected_index = index

    # Return the selected individual
    return population[selected_index]

In [8]:
def crossover(bounds, parent1, parent2, variance):
    """
    Perform crossover between two parents and add variance to the children.

    Args:
        parent1 (list): A list of two values representing the mass and CG of the first parent.
        parent2 (list): A list of two values representing the mass and CG of the second parent.
        variance (float): The amount of variance to add to the children.
        bounds (list): Bounds for the input data, where each element is a tuple (min, max) for a dimension..

    Returns:
        tuple: A tuple of two lists representing the two new children.
    """
    # Consider the bounds on mass and CG
    mass_min,mass_max,cg_min,cg_max = bounds[0][0],bounds[0][1],bounds[1][0],bounds[1][1]
    
    # Calculate the mean of the mass and CG values of the two parents
    mass_mean = (parent1[0] + parent2[0]) / 2
    cg_mean = (parent1[1] + parent2[1]) / 2
    
    # Create the first child by combining the first part of parent1 with the second part of parent2
    child1 = [parent1[0], cg_mean]
    child1[0] += random.uniform(-variance, variance)
    child1[1] += random.uniform(-variance, variance)
    
    # Check if the mass and CG values of the first child are within bounds
    if child1[0] < mass_min:
        child1[0] = mass_min
    elif child1[0] > mass_max:
        child1[0] = mass_max
    if child1[1] < cg_min:
        child1[1] = cg_min
    elif child1[1] > cg_max:
        child1[1] = cg_max
    
    # Create the second child by combining the first part of parent2 with the second part of parent1
    child2 = [mass_mean, parent2[1]]
    child2[0] += random.uniform(-variance, variance)
    child2[1] += random.uniform(-variance, variance)
    
    # Check if the mass and CG values of the second child are within bounds
    if child2[0] < mass_min:
        child2[0] = mass_min
    elif child2[0] > mass_max:
        child2[0] = mass_max
    if child2[1] < cg_min:
        child2[1] = cg_min
    elif child2[1] > cg_max:
        child2[1] = cg_max
    
    # Return the new children as a tuple
    return child1, child2

In [9]:
def mutate(bounds, individual, mutation_rate, mutation_variance):
    """
    Mutate an individual by randomly perturbing its values.
    
    Args:
    individual (list): The individual to mutate.
    mutation_rate (float): The probability of mutation occurring.
    mutation_variance (float): The maximum amount of perturbation to add.
    
    Return: The mutated individual.
    """
    # Consider the bounds on mass and CG
    mass_min,mass_max,cg_min,cg_max = bounds[0][0],bounds[0][1],bounds[1][0],bounds[1][1]
    
    # Check if mutation should occur
    if random.random() < mutation_rate:
        
        # Add random perturbation to the mass value
        individual[0] += random.uniform(-mutation_variance, mutation_variance)
        
        # Add random perturbation to the CG value
        individual[1] += random.uniform(-mutation_variance, mutation_variance)
        
        # Check bounds for mass
        if individual[0] < mass_min:
            individual[0] = mass_min
        elif individual[0] > mass_max:
            individual[0] = mass_max
            
        # Check bounds for CG
        if individual[1] < cg_min:
            individual[1] = cg_min
        elif individual[1] > cg_max:
            individual[1] = cg_max
    
    return individual

In [12]:
#____________________MAIN FUNCTION_____________________________________

# Grid boundaries for inputs
bounds = [[30000, 40000], [25, 45]]  #limits on each parameter - mass_min,mass_max,cg_min,cg_max
step = 3 #number of points in each parameter

# define the population size
n_pop = 10
# define the number of generations
n_gen = 10
# selection number 
n_sel = 6
# crossover rate
r_cross = 0.5
# crossover variance
variance = 1;
# mutation rate
r_mut = 0.2    
# mutation variance


# Generate input data
input = initial_population(bounds,step)

# Generate simulation data 
output = simulation_data(input,step) 

# make a dictionary of the simulated data
sim_data = overall_data(input,output)

# Iterate for Generations
for i in range(n_gen-1):
    separate_input_data(sim_data)

 




NameError: name 'best' is not defined